In [1]:
import torch.nn as nn
import torch

In [2]:
#  test indice of torch
x = torch.randn(3, 4, 3)
x[:,1,:].shape

torch.Size([3, 3])

In [89]:
# tensornetwork LM

class TN(nn.Module):
    def __init__(self, rank = 10,vocab_size = 1000):
        super().__init__()
        self.r = rank
        self.G = nn.Embedding(vocab_size,self.r * self.r)
        self.G1 = nn.Linear(1,self.r)
        self.linear = torch.nn.Linear(self.r,1)
        self.vocab_size = vocab_size
    def encoding(self,x):
        batch_size,seq_length = x.shape[0],x.shape[1]
        self.masked_token = self.G.weight.view(-1,self.vocab_size,self.r,self.r)
        encoded = self.G(x)
        encoded = encoded.view(batch_size, seq_length,self.r,self.r)
        result = self.G1.weight.view(-1,self.r).unsqueeze(0).repeat([batch_size,1,1])
        for i in range(seq_length):
            if x[:,i] == 103:
                result = torch.einsum("bij,bvjr->bvr",[result,self.masked_token])
            else:
                result = torch.nn.functional.normalize(torch.einsum("bij,bjk->bik",[result,encoded[:,i,:,:]]))
        return result.squeeze()

    def forward(self,x):
        encoded_x = self.encoding(x)
        return self.linear(encoded_x).squeeze()

    def get_structure_penalty(self): # unitary or l2 loss
        embedding = self.encoder.weight.view(-1,self.d,self.d)
        embedding_transpose = embedding.transpose(-1,-2)
        product = torch.bmm(embedding_transpose,embedding)
        diff = torch.abs(product.mean(0).squeeze() -torch.eye(self.d,device=torch.device("cuda" if torch.cuda.is_available() else "cpu")))
        return torch.sum(diff)

In [90]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
x = torch.tensor([1,2,3,103,5]).unsqueeze(0).to(device)
x.shape

model = TN(vocab_size = 1000,rank = 10)

# and move our model over to the selected device
model = model.cuda()

result = model(x)
print(result.shape)

torch.Size([1000])


In [1]:
# #  test nn.embedding, it is G in our paper, make G is shared.
# vocab_size = 1000
# rank = 5
# batch_size = 1
# G = nn.Embedding(vocab_size,rank * rank)

# masked_token = G.weight.view(-1,vocab_size,rank,rank)
# print(masked_token.shape)

# # G1 is a vecetor
# G_1 = nn.Linear(1,rank)

# # input x
# x = torch.tensor([1,2,3,4,5])

# embedding_x = G(x)
# # reshape the embedding
# embedding_x = embedding_x.view(-1,len(x),rank,rank)
# print(embedding_x.shape)

# # masked token
# masked_index = 2
# x[masked_index] = -100

# print("x;",x)

# init_matrix = G_1.weight.view(-1,rank).unsqueeze(0).repeat([batch_size,1,1])

# print(init_matrix.shape)

# torch.bmm(init_matrix,embedding_x[:,1,:,:])

# mask_mul = torch.einsum("bij,bvjr->bvr",[init_matrix,masked_token])
# print("mask_mul",mask_mul.shape)
# # test a sequence, assume a token is masked in our sentence. 

# result = init_matrix

# print("result:",result.shape)
# for i in range(len(x)):
#     if x[i] == -100:
#         result = torch.einsum("bij,bvjr->bvr",[result,masked_token])
#     else:
#         result = torch.einsum("bij,bjk->bik",[result,embedding_x[:,i,:,:]])
# print(result.shape)

In [6]:
# 训练 tensor network language model

from transformers import AutoConfig,AutoTokenizer
config = AutoConfig.from_pretrained("bert-base-uncased")
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
print(config.vocab_size)



30522


In [7]:
with open("lm_data/clean.txt",'r') as fp:
    text = fp.read().split("\n")

text[:5]

['From my grandfather Verus I learned good morals and the government of my temper.',
 'From the reputation and remembrance of my father, modesty and a manly character.',
 'From my mother, piety and beneficence, and abstinence, not only from evil deeds, but even from evil thoughts; and further, simplicity in my way of living, far removed from the habits of the rich.',
 'From my great-grandfather, not to have frequented public schools, and to have had good teachers at home, and to know that on such things a man should spend liberally.',
 "From my governor, to be neither of the green nor of the blue party at the games in the Circus, nor a partizan either of the Parmularius or the Scutarius at the gladiators' fights; from him too I learned endurance of labour, and to want little, and to work with my own hands, and not to meddle with other people's affairs, and not to be ready to listen to slander."]

In [9]:
inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
inputs

Ignored unknown kwarg option direction


{'input_ids': tensor([[  101,  2013,  2026,  ...,     0,     0,     0],
        [  101,  2013,  1996,  ...,     0,     0,     0],
        [  101,  2013,  2026,  ...,     0,     0,     0],
        ...,
        [  101,  2122,  2093,  ...,     0,     0,     0],
        [  101,  3459,  2185,  ...,     0,     0,     0],
        [  101,  2043, 15223,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [16]:
import random
selection = random.sample(range(1, 512), inputs.input_ids.shape[0])
selection[:5]

[404, 81, 442, 139, 26]

In [42]:
inputs['labels'] = torch.tensor(selection)
inputs.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

In [43]:
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103
# for i in inputs.input_ids:
#     print(list(i).index(103))

In [44]:
import torch
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

In [45]:
dataset = MeditationsDataset(inputs)


In [46]:
loader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=True)

In [91]:
model = TN(vocab_size = config.vocab_size)

In [92]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)
# activate training mode
model.train()

TN(
  (G): Embedding(30522, 100)
  (G1): Linear(in_features=1, out_features=10, bias=True)
  (linear): Linear(in_features=10, out_features=1, bias=True)
)

In [41]:
from transformers import AdamW
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

/home/wzm289/miniconda3/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [94]:
from tqdm import tqdm  # for our progress bar
from torch.nn import CrossEntropyLoss
epochs = 2
for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        # attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        logits = model(input_ids)
        
        loss_fct = CrossEntropyLoss()  # -100 index = padding token
        print(logits)
        masked_lm_loss = loss_fct(logits.view(-1, model.vocab_size), labels.view(-1))
        # extract loss
        loss = masked_lm_loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

  0%|          | 0/506 [00:00<?, ?it/s]/tmp/ipykernel_1379340/1368166411.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


tensor([0.1953, 0.1971, 0.1990,  ..., 0.1955, 0.1964, 0.1977], device='cuda:0',
       grad_fn=<SqueezeBackward0>)


Epoch 0:   0%|          | 1/506 [00:00<03:40,  2.29it/s, loss=10.3]

tensor([0.1977, 0.1957, 0.1952,  ..., 0.1957, 0.1974, 0.1944], device='cuda:0',
       grad_fn=<SqueezeBackward0>)


Epoch 0:   0%|          | 2/506 [00:00<03:44,  2.25it/s, loss=10.3]

...

tensor([0.1972, 0.1954, 0.1959,  ..., 0.1968, 0.1958, 0.1960], device='cuda:0',
       grad_fn=<SqueezeBackward0>)


Epoch 0:  59%|█████▉    | 300/506 [02:24<01:37,  2.11it/s, loss=10.3]

tensor([0.1955, 0.1965, 0.1999,  ..., 0.1962, 0.1944, 0.2003], device='cuda:0',
       grad_fn=<SqueezeBackward0>)


Epoch 0:  59%|█████▉    | 301/506 [02:24<01:38,  2.08it/s, loss=10.3]


KeyboardInterrupt: 